In [10]:
import os
import torch
import importlib
import glob
from seqexplainer import _layer_outs as activations

# Load a model

## Where else could I load from?

## A EUGENe model

In [15]:
from eugene import models
from eugene import settings
settings.logging_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/logs/revision/jores21"
settings.config_dir = "/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/configs/jores21"

In [16]:
leaf_model_file = glob.glob(os.path.join(settings.logging_dir, "cnn", "leaf_trial_3", "checkpoints", "*"))[0]
model = models.load_config(config_path="cnn.yaml")
leaf_model = models.SequenceModule.load_from_checkpoint(leaf_model_file, arch=model.arch)

[rank: 0] Global seed set to 3


In [17]:
### NEED TO GET SOMETHING LIKE THIS WORKING
def _load_eugene_model(model_path):
    model_state = torch.load(model_path)
    try:
        arch = model_state["hyper_parameters"]["arch"]
        model_type = getattr(importlib.import_module("eugene.models"), arch)
        model = model_type(**model_state["hyper_parameters"])
        model.load_state_dict(model_state["state_dict"])
    except ImportError:
        raise ImportError("Please install eugene-tools (pip install eugene-tools) to use this feature.")
    return model
#model = _load_eugene_model('/cellar/users/aklie/projects/ML4GLand/EUGENe_paper/logs/revision/jores21/cnn/leaf_trial_2/checkpoints/epoch=6-step=3206.ckpt')

# Get a layer

In [21]:
# Get list of available layers
available_layers = activations.list_available_layers(model, key_word="conv1d_tower")
available_layers

AttributeError: module 'seqexplainer._activations' has no attribute 'list_available_layers'

In [24]:
# Select a layer
layer_name = "arch.conv1d_tower.layers.0"

In [25]:
# Actually get the layer
layer = activations.get_layer(
    model=model,
    layer_name=layer_name
)
layer

Conv1d(4, 256, kernel_size=(13,), stride=(1,), padding=valid)

# Get layer outputs

In [26]:
inputs = torch.randn(10, 4, 100)

In [27]:
activations.get_layer_outputs(
    model=model,
    inputs=inputs,
    layer_name=layer_name,
    batch_size=128,
    device="cuda"   
)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
